# Overview: determine a correlation between baseball salary and various metrics, specifically regarding correlation between advanced metrics and simple metrics for salary prediction

Process expectation:

    1 - Build database storage for historical database
        a - master storage in sqlite
        b - year by year storage in csv, then batch update master storage

    2 - Scrape using Selenium
        a - learning Selenium over requests + bs4
        b - baseball-reference.com for stats including salary database

    3 - Build stats and analytics
        a - specifics to come later
        

In [2]:
# Configuration and setup - RUN THIS FIRST

# Globals
DB_NAME = 'database.db'
SCRAPES_PER_MINUTE = 20

# Selenium config
HEADLESS = False
DRIVER_LOCATION = 'chromedriver.exe'

# Imports
from playerData import PlayerData
from sqlite3 import connect

# Setup
conn = connect(DB_NAME)

In [3]:
# Selenium initialization
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options

chrome_options = Options()
if HEADLESS:
    chrome_options.add_argument("--headless")  # optional for headless mode
chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                            "AppleWebKit/537.36 (KHTML, like Gecko) "
                            "Chrome/115.0.0.0 Safari/537.36")
chrome_options.add_argument("--window-size=1000,720")

DRIVER = webdriver.Chrome(service=Service(DRIVER_LOCATION),
                          options=chrome_options)

# Quick test
DRIVER.get("https://www.google.com")
print(DRIVER.title)


Google


In [3]:
# Building the database

from sqlite3 import Connection

def initialize_database(conn: Connection):
    create_summary_table(conn)
    create_standard_batting_table(conn)
    create_advanced_batting_table(conn)
    create_value_batting_table(conn)
    create_standard_pitching_table(conn)
    create_advanced_pitching_table(conn)
    create_value_pitching_table(conn)
    create_standard_fielding_table(conn)
    create_salary_table(conn)
    create_play_years_table(conn)


def create_table_from_contract(conn: Connection, name: str, columns: dict[str, str]):
    table_string = f'CREATE TABLE IF NOT EXISTS {name} ('

    for key in columns:
        table_string += f'\n{key} {columns[key]},'
    
    # drop last comma and close
    table_string = table_string[:-1]
    table_string += ');'
    
    cursor = conn.cursor()
    cursor.execute(table_string)
    cursor.close()

def create_summary_table(conn:Connection):
    
    table_dict = {
            "composite_id": "TEXT PRIMARY KEY",
            "first_name": "TEXT",
            "last_name": "TEXT",
            "team": "TEXT",
            "league": "TEXT",
            "bats": "TEXT",
            "throws": "TEXT",
            "height_inches": "INTEGER",
            "weight_lbs": "INTEGER",
            "recent_headshot": "BLOB",
            "birthday": "TEXT",
            "hometown": "TEXT",
            "awards": "TEXT"
        }
    
    create_table_from_contract(conn, 'summary', table_dict)

def create_standard_batting_table(conn:Connection):
    
    table_dict = {
            "composite_id": "TEXT PRIMARY KEY",
            "wins_above_replacement": "REAL",
            "games": "INTEGER",
            "plate_appearances": "INTEGER",
            "at_bats": "INTEGER",
            "runs_scored": "INTEGER",
            "hits": "INTEGER",
            "doubles": "INTEGER",
            "triples": "INTEGER",
            "home_runs": "INTEGER",
            "runs_batted_in": "INTEGER",
            "stolen_bases": "INTEGER",
            "caught_stealing": "INTEGER",
            "walks": "INTEGER",
            "strike_outs": "INTEGER",
            "batting_average": "REAL",
            "on_base_percentage": "REAL",
            "slugging": "REAL",
            "ops": "REAL",
            "ops_plus": "REAL",
            "roba": "REAL",
            "rbat_plus": "REAL",
            "total_bases": "INTEGER",
            "grounded_into_double_plays": "INTEGER",
            "hit_by_pitch": "INTEGER",
            "sacrifice_bunts": "INTEGER",
            "sacrifice_flies": "INTEGER",
            "intentional_walks": "INTEGER",
            "position": "TEXT",
        }
    
    create_table_from_contract(conn, 'standard_batting', table_dict)

def create_advanced_batting_table(conn:Connection):

    table_dict = {

            "composite_id": "TEXT PRIMARY KEY",
            "babip": "REAL",
            "iso": "REAL",
            "hr_pct": "REAL",
            "so_pct": "REAL",
            "bb_pct": "REAL",
            "ev": "REAL",
            "hard_hit_pct": "REAL",
            "ld_pct": "REAL",
            "gb_pct": "REAL",
            "fb_pct": "REAL",
            "gb_fb_ratio": "REAL",
            "pull_pct": "REAL",
            "center_pct": "REAL",
            "oppo_pct": "REAL",
            "wpa": "REAL",
            "cwpa": "REAL",
            "re24": "REAL",
            "rs_pct": "REAL",
            "sb_pct": "REAL",
            "xbt_pct": "REAL",
        }
    
    create_table_from_contract(conn, 'advanced_batting', table_dict)    

def create_value_batting_table(conn:Connection):
    
    table_dict = {"composite_id": "TEXT PRIMARY KEY",          # unique player ID
            "PA": "REAL",                     # plate appearances
            "Rbat": "REAL",                   # runs from batting
            "Rbaser": "REAL",                 # runs from baserunning
            "Rdp": "REAL",                     # runs lost to double plays
            "Rfield": "REAL",                 # runs from fielding
            "Rpos": "REAL",                    # positional adjustment runs
            "RAA": "REAL",                     # runs above average
            "WAA": "REAL",                     # wins above average
            "Rrep": "REAL",                    # replacement runs
            "RAR": "REAL",                      # runs above replacement
            "WAR": "REAL",                      # wins above replacement
            "waa_wl_pct": "REAL",              # WAA WL%
            "wl_162": "REAL",                  # 162WL%
            "oWAR": "REAL",                     # offensive WAR
            "dWAR": "REAL",                     # defensive WAR
            "oRAR": "REAL",                     # offensive RAR
        }
    
    create_table_from_contract(conn, 'value_batting', table_dict)  

def create_standard_pitching_table(conn:Connection):
    table_dict = {
            "composite_id": "TEXT PRIMARY KEY",
            "wins": "INTEGER",
            "losses": "INTEGER",
            "win_loss_pct": "REAL",
            "era": "REAL",
            "games": "INTEGER",
            "games_started": "INTEGER",
            "games_finished": "INTEGER",
            "complete_games": "INTEGER",
            "shutouts": "INTEGER",
            "saves": "INTEGER",
            "innings_pitched": "REAL",
            "hits_allowed": "INTEGER",
            "runs_allowed": "INTEGER",
            "earned_runs": "INTEGER",
            "home_runs_allowed": "INTEGER",
            "walks": "INTEGER",
            "intentional_walks": "INTEGER",
            "strike_outs": "INTEGER",
            "hit_by_pitch": "INTEGER",
            "balks": "INTEGER",
            "wild_pitches": "INTEGER",
            "batters_faced": "INTEGER",
            "era_plus": "REAL",
            "fip": "REAL",
            "whip": "REAL",
            "hits_per_9": "REAL",
            "hr_per_9": "REAL",
            "bb_per_9": "REAL",
            "so_per_9": "REAL",
            "so_to_bb": "REAL",
    }
    
    create_table_from_contract(conn, 'standard_pitching', table_dict)  

def create_advanced_pitching_table(conn:Connection):
    table_dict = {
            "composite_id": "TEXT PRIMARY KEY",
            "innings_pitched": "REAL",       # IP
            "batting_average": "REAL",       # BA
            "on_base_percentage": "REAL",    # OBP
            "slugging": "REAL",              # SLG
            "ops": "REAL",                   # OPS
            "babip": "REAL",                 # BAbip
            "hr_pct": "REAL",                # HR%
            "k_pct": "REAL",                 # K%
            "bb_pct": "REAL",                # BB%
            "exit_velocity": "REAL",         # EV
            "hard_hit_pct": "REAL",          # HardH%
            "ld_pct": "REAL",                # LD%
            "gb_pct": "REAL",                # GB%
            "fb_pct": "REAL",                # FB%
            "gb_fb_ratio": "REAL",           # GB/FB
            "wpa": "REAL",                   # WPA
            "cwpa": "REAL",                  # cWPA
            "re24": "REAL",                  # RE24
        }
    create_table_from_contract(conn, 'advanced_pitching', table_dict)  

def create_value_pitching_table(conn:Connection):
    table_dict = {
            "composite_id": "TEXT PRIMARY KEY",
            "innings_pitched": "REAL",       # IP
            "games": "INTEGER",              # G
            "games_started": "INTEGER",      # GS
            "runs_allowed": "INTEGER",       # R
            "ra9": "REAL",                   # RA9
            "ra9_opponent": "REAL",          # RA9opp
            "ra9_defense": "REAL",           # RA9def
            "ra9_role": "REAL",              # RA9role
            "ra9_extras": "REAL",            # RA9extras
            "ppfp": "REAL",                  # PPFp
            "ra9_avg": "REAL",               # RA9avg
            "raa": "REAL",                    # RAA
            "waa": "REAL",                    # WAA
            "waa_adj": "REAL",                # WAAadj
            "war": "REAL",                    # WAR
            "rar": "REAL",                    # RAR
            "waa_wl_pct": "REAL",             # WAA WL%
            "wl_162": "REAL",                 # 162WL%
        }
    
    create_table_from_contract(conn, 'value_pitching', table_dict)  

def create_standard_fielding_table(conn:Connection):
    table_dict = {
            "composite_id": "TEXT PRIMARY KEY",
            "position": "TEXT",                  # Pos
            "games": "INTEGER",                  # G
            "games_started": "INTEGER",          # GS
            "complete_games": "INTEGER",         # CG
            "innings": "REAL",                   # Inn
            "chances": "INTEGER",                # Ch
            "putouts": "INTEGER",                # PO
            "assists": "INTEGER",                # A
            "errors": "INTEGER",                 # E
            "double_plays": "INTEGER",           # DP
            "fielding_percentage": "REAL",       # Fld%
            "league_fielding_percentage": "REAL",# lgFld%
            "total_zone_total": "REAL",          # Rtot
            "total_zone_per_year": "REAL",       # Rtot/yr
            "defensive_runs_saved": "REAL",      # Rdrs
            "defensive_runs_saved_per_year": "REAL", # Rdrs/yr
            "range_factor_per_9": "REAL",        # RF/9
            "league_range_factor_per_9": "REAL", # lgRF9
            "range_factor_per_game": "REAL",     # RF/G
            "league_range_factor_per_game": "REAL", # lgRFG
            "stolen_bases_allowed": "INTEGER",   # SB
            "caught_stealing": "INTEGER",        # CS
            "caught_stealing_percentage": "REAL",# CS%
            "league_caught_stealing_percentage": "REAL", # lgCS%
            "pickoffs": "INTEGER",               # Pick
        }
    
    create_table_from_contract(conn, 'standard_fielding', table_dict)  

def create_salary_table(conn:Connection):
    table_dict = {
            "composite_id": "TEXT PRIMARY KEY",
            "year": "INTEGER",    # Year
            "salary": "REAL",     # Salary
        }
    
    create_table_from_contract(conn, 'salary', table_dict)  

def create_play_years_table(conn:Connection):
    query = '''CREATE TABLE IF NOT EXISTS player_years (
    composite_id TEXT NOT NULL,
    year INTEGER NOT NULL,
    hall_of_fame INTEGER,
    PRIMARY KEY (composite_id, year)
    );'''
    
    conn.execute(query)

initialize_database(conn)

In [4]:
# Playing years scraping

import sqlite3
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from collections import deque
from datetime import datetime
import time

def wait_for_rate_limit(request_times: deque, limit: int = 20, interval: int = 60):
    now = datetime.now()
    if len(request_times) == limit:
        oldest = request_times[0]
        elapsed = (now - oldest).total_seconds()
        if elapsed < interval:
            print('Waiting')
            time.sleep(interval - elapsed)
            print('Wait complete')
    request_times.append(datetime.now())

def insert_players_years_played(conn: sqlite3.Connection, player_data: list[dict]):
    query = """
    INSERT OR IGNORE INTO player_years (composite_id, year, hall_of_fame)
    VALUES (:composite_id, :year, :hall_of_fame)
    """
    conn.executemany(query, player_data)
    conn.commit()

def process_page(letter: str):
    def pull_composite_id(url: str):
        return url.rsplit('/', 1)[-1].split('.')[0]
    
    def pull_years_and_hof(text: str):
        hall_of_fame = 1 if '+' in text else 0
        years = text.replace('+', '').split('(')[1].rstrip(')')
        first_year, last_year = map(int, years.split('-'))
        return hall_of_fame, first_year, last_year
    
    DRIVER.get(f'https://www.baseball-reference.com/players/{letter}/')
    players_div = WebDriverWait(DRIVER, 10, poll_frequency=0.25).until(
    EC.presence_of_element_located((By.ID, "div_players_"))
)
    player_paragraphs = players_div.find_elements(By.TAG_NAME, "p")

    all_data = []

    for p in player_paragraphs:
        link = p.find_element(By.TAG_NAME, "a").get_attribute("href")
        if not link: # this should never happen
            raise ValueError("Expecting href and found none")
        composite_id = pull_composite_id(link)

        text = p.text
        hall_of_fame, first_year, last_year = pull_years_and_hof(text)

        data = [
            {'composite_id': composite_id,
             'year': year,
             'hall_of_fame': hall_of_fame}
            for year in range(first_year, last_year + 1)
        ]
        all_data.extend(data)

    insert_players_years_played(conn, all_data)


request_times = deque(maxlen=20)  # store the timestamps of the last 20 requests
alphabet = list("abcdefghijklmnopqrstuvwxyz")
for letter in alphabet:
    print(f"Scraping {letter}")
    wait_for_rate_limit(request_times, SCRAPES_PER_MINUTE)
    process_page(letter)
    print(f"Scraping {letter} complete")

DRIVER.quit()
    


Scraping a
Scraping a complete
Scraping b
Scraping b complete
Scraping c
Scraping c complete
Scraping d
Scraping d complete
Scraping e
Scraping e complete
Scraping f
Scraping f complete
Scraping g
Scraping g complete
Scraping h
Scraping h complete
Scraping i
Scraping i complete
Scraping j
Scraping j complete
Scraping k
Scraping k complete
Scraping l
Scraping l complete
Scraping m
Scraping m complete
Scraping n
Scraping n complete
Scraping o
Scraping o complete
Scraping p
Scraping p complete
Scraping q
Scraping q complete
Scraping r
Scraping r complete
Scraping s
Scraping s complete
Scraping t
Scraping t complete
Scraping u
Scraping u complete
Scraping v
Scraping v complete
Scraping w
Scraping w complete
Scraping x
Scraping x complete
Scraping y
Scraping y complete
Scraping z
Scraping z complete


In [ ]:
# Batch storage creation

import csv
from dataclasses import fields
from pathlib import Path
from playerData import (
        Summary, StandardBatting, AdvancedBatting, ValueBatting,
        StandardPitching, AdvancedPitching, ValuePitching,
        StandardFielding, SalaryEntry
    )

def write_table_headers_to_csv(output_dir):
    table_classes = {
        "summary": Summary,
        "standard_batting": StandardBatting,
        "advanced_batting": AdvancedBatting,
        "value_batting": ValueBatting,
        "standard_pitching": StandardPitching,
        "advanced_pitching": AdvancedPitching,
        "value_pitching": ValuePitching,
        "standard_fielding": StandardFielding,
        "salary_history": SalaryEntry,
    }

    output_path = Path(output_dir)
    output_path.mkdir(parents=True, exist_ok=True)

    for table_name, cls in table_classes.items():
        csv_file = output_path / f"{table_name}.csv"

        if csv_file.exists():
            print(f"Skipped {table_name}, file already exists: {csv_file}")
            continue

        header_fields = [f.name for f in fields(cls)]
        
        with open(csv_file, mode="w", newline="", encoding="utf-8") as f:
            writer = csv.writer(f)
            writer.writerow(header_fields)
        
        print(f"Wrote headers for {table_name} to {csv_file}")

for year in range(START_YEAR, END_YEAR + 1):
    write_table_headers_to_csv(f'data/{year}')






Wrote headers for summary to data\1990\summary.csv
Wrote headers for standard_batting to data\1990\standard_batting.csv
Wrote headers for advanced_batting to data\1990\advanced_batting.csv
Wrote headers for value_batting to data\1990\value_batting.csv
Wrote headers for standard_pitching to data\1990\standard_pitching.csv
Wrote headers for advanced_pitching to data\1990\advanced_pitching.csv
Wrote headers for value_pitching to data\1990\value_pitching.csv
Wrote headers for standard_fielding to data\1990\standard_fielding.csv
Wrote headers for salary_history to data\1990\salary_history.csv
Wrote headers for summary to data\1991\summary.csv
Wrote headers for standard_batting to data\1991\standard_batting.csv
Wrote headers for advanced_batting to data\1991\advanced_batting.csv
Wrote headers for value_batting to data\1991\value_batting.csv
Wrote headers for standard_pitching to data\1991\standard_pitching.csv
Wrote headers for advanced_pitching to data\1991\advanced_pitching.csv
Wrote heade